Demo ipynb for CTM (Contextualized Topic Models)

Since we aren't doing multi-language, we may use the Combined TM instead of zero-shot one

In [1]:
import pandas as pd
import numpy as np


from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
# from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessingStopwords

import nltk

from pathlib import Path

/Users/michaelcheng/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset_path = Path('../../dataset/topic_modelling/top_10_games/00_Terraria.pkl')

dataset = pd.read_pickle(dataset_path)

dataset.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 75499 entries, 57735 to 133233
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   index         75499 non-null  int64 
 1   app_id        75499 non-null  int64 
 2   app_name      75499 non-null  object
 3   review_text   75499 non-null  object
 4   review_score  75499 non-null  int64 
 5   review_votes  75499 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 4.0+ MB


In [6]:
%load_ext autoreload

In [7]:
# data preprocessing

import sys
sys.path.append('../../sa/')

%autoreload 2
import str_cleaning_functions

# copied from lda_demo_gridsearch.ipynb
def cleaning(df, review):
    df[review] = df[review].apply(lambda x: str_cleaning_functions.remove_links(x))
    df[review] = df[review].apply(lambda x: str_cleaning_functions.remove_links2(x))
    df[review] = df[review].apply(lambda x: str_cleaning_functions.clean(x))
    df[review] = df[review].apply(lambda x: str_cleaning_functions.deEmojify(x))
    df[review] = df[review].apply(lambda x: str_cleaning_functions.remove_non_letters(x))
    df[review] = df[review].apply(lambda x: x.lower())
    df[review] = df[review].apply(lambda x: str_cleaning_functions.unify_whitespaces(x))
    df[review] = df[review].apply(lambda x: str_cleaning_functions.remove_stopword(x))
    df[review] = df[review].apply(lambda x: str_cleaning_functions.unify_whitespaces(x))

# def cleaning_strlist(str_list):
#     str_list = list(map(lambda x: clean(x), str_list))
#     str_list = list(map(lambda x: deEmojify(x), str_list))

#     str_list = list(map(lambda x: x.lower(), str_list))
#     str_list = list(map(lambda x: remove_num(x), str_list))
#     str_list = list(map(lambda x: unify_whitespaces(x), str_list))

#     str_list = list(map(lambda x: _deaccent(x), str_list))
#     str_list = list(map(lambda x: remove_non_alphabets(x), str_list))
#     str_list = list(map(lambda x: remove_stopword(x), str_list))
#     return str_list

# copied from bert_demo_gridsearch.ipynb
def cleaning_little(df, review):
    df[review] = df[review].apply(lambda x: str_cleaning_functions.remove_links(x))
    df[review] = df[review].apply(lambda x: str_cleaning_functions.remove_links2(x))
    df[review] = df[review].apply(lambda x: str_cleaning_functions.clean(x))
    df[review] = df[review].apply(lambda x: str_cleaning_functions.deEmojify(x))
    df[review] = df[review].apply(lambda x: str_cleaning_functions.unify_whitespaces(x))


In [8]:
# create a copy of the dataset, as we need both untouched text and cleaned text

dataset_preprocessed = dataset.copy()

In [9]:
cleaning(dataset_preprocessed, 'review_text')


cleaning_little(dataset, 'review_text')

In [10]:
X_preprocessed = dataset_preprocessed['review_text'].values
X = dataset['review_text'].values

Apply lemmatizing to the preprocessed dataset as well (for BoW)

In [11]:
# do lemmatization, but not stemming (as part of speech is important in topic modelling)
# use nltk wordnet for lemmatization

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

lemma = WordNetLemmatizer()

# from https://stackoverflow.com/questions/25534214/nltk-wordnet-lemmatizer-shouldnt-it-lemmatize-all-inflections-of-a-word

# from: https://www.cnblogs.com/jclian91/p/9898511.html
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None     # if none -> created as noun by wordnet
    
def lemmatization(text):
   # use nltk to get PoS tag
    tagged = nltk.pos_tag(nltk.word_tokenize(text))

    # then we only need adj, adv, verb, noun
    # convert from nltk Penn Treebank tag to wordnet tag
    wn_tagged = list(map(lambda x: (x[0], get_wordnet_pos(x[1])), tagged))

    # lemmatize by the PoS
    lemmatized = list(map(lambda x: lemma.lemmatize(x[0], pos=x[1] if x[1] else wordnet.NOUN), wn_tagged))
    # lemma.lemmatize(wn_tagged[0], pos=wordnet.NOUN)

    return lemmatized

In [12]:
X_preprocessed = list(map(lambda x: lemmatization(x), X_preprocessed))
X_preprocessed = list(map(lambda x: ' '.join(x), X_preprocessed))

reduce the BoW to 2000 as recommended by CTM

In [13]:
# referencing to preprocessing in CTM
# to keep first 2000 words in the preprpcessed documents

from sklearn.feature_extraction.text import CountVectorizer

VOCABULARY_SIZE = 2000

vectorizer = CountVectorizer(max_features=VOCABULARY_SIZE)
vectorizer.fit_transform(X_preprocessed)
temp_vocabulary = set(vectorizer.get_feature_names_out())

preprocessed_docs_tmp = [' '.join([w for w in doc.split() if w in temp_vocabulary])
                            for doc in X_preprocessed]

X_preprocessed = preprocessed_docs_tmp

In [14]:
X_preprocessed[0]

'werewolf rid unicorn rainbow gun build find hair spider cavern get sword shoot cat take lord moon use yoyo summon minion shoot enemy find sky temple air spawn buy music box wizard go record music like play base whenever want go build castle make entirely white would seem thing minecraft game dimension trust get use start game terrarium simply one satisfy sandbox experience may sound compare minecraft imagination'

Training

In [22]:
# copy from: https://github.com/MilaNLProc/contextualized-topic-models/blob/master/contextualized_topic_models/utils/data_preparation.py#L44
# call bert_embeddings_from_list() to produce embeddings by ourself

import warnings
from sentence_transformers import SentenceTransformer
import torch
import platform


if platform.system() == 'Linux' or platform.system() == 'Windows':
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
else:
    device = torch.device('mps')        # m-series mac machine

print(device)

def bert_embeddings_from_list(
    texts, sbert_model_to_load, batch_size=200, max_seq_length=128, device='cpu'):      # 128 is the default valule in TopicModelDataPreparation()
    """
    Creates SBERT Embeddings from a list
    """

    model = SentenceTransformer(sbert_model_to_load, device=device)

    if max_seq_length is not None:
        model.max_seq_length = max_seq_length

    check_max_local_length(max_seq_length, texts)

    return np.array(model.encode(texts, show_progress_bar=True))


def check_max_local_length(max_seq_length, texts):
    max_local_length = np.max([len(t.split()) for t in texts])
    if max_local_length > max_seq_length:
        warnings.simplefilter("always", DeprecationWarning)
        warnings.warn(
            f"the longest document in your collection has {max_local_length} words, the model instead "
            f"truncates to {max_seq_length} tokens."
        )

mps


In [23]:
tp = TopicModelDataPreparation()        # we create the sbert embeddings by ourself (so that m1 machines can create embeddings using gpu)

X_contextualized = bert_embeddings_from_list(X, "all-mpnet-base-v2", device=device)

training_dataset = tp.fit(text_for_contextual=X, text_for_bow=X_preprocessed, custom_embeddings=X_contextualized)

/var/folders/hm/n8540hpn7xd1f0pw3_qnsp4r0000gn/T/ipykernel_24817/434925383.py:37: UserWarning: the longest document in your collection has 1987 words, the model instead truncates to 128 tokens.
  warnings.warn(
Batches: 100%|██████████| 2360/2360 [03:16<00:00, 11.99it/s]


In [52]:
training_dataset.X_bow.shape

(75499, 2000)

In [26]:
X_contextualized.shape

(75499, 768)

In [27]:
# check the vocabs

tp.vocab[:10]

array(['aaa', 'abandon', 'ability', 'able', 'absolute', 'absolutely',
       'absolutly', 'accesories', 'access', 'accessory'], dtype=object)

In [28]:
len(tp.vocab)

2000

Training combined TM

In [30]:
N_TOPICS = 20

ctm = CombinedTM(bow_size=len(tp.vocab), contextual_size=768, n_components=N_TOPICS, num_epochs=10)
ctm.device = device     # force using gpu for m1 machines
ctm.fit(training_dataset, verbose=True) # run the model

0it [00:00, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [31]:
ctm.get_topic_lists(5)

[['game', 'great', 'content', 'hour', 'terrarium'],
 ['worth', 'content', 'update', 'keep', 'add'],
 ['minecraft', 'like', 'terrarium', 'say', 'much'],
 ['pretty', 'good', 'nice', 'cool', 'guess'],
 ['fix', 'please', 'mac', 'cant', 'wont'],
 ['would', 'kill', 'die', 'slime', 'guide'],
 ['best', 'ever', 'one', 'favorite', 'ive'],
 ['hour', 'play', 'time', 'still', 'come'],
 ['addictive', 'addict', 'super', 'list', 'mostly'],
 ['terrarium', 'game', 'minecraft', 'world', 'build'],
 ['awesome', 'love', 'amaze', 'recommend', 'highly'],
 ['minecraft', 'like', 'well', 'terrarium', 'terraria'],
 ['build', 'explore', 'boss', 'fight', 'world'],
 ['list', 'across', 'chance', 'felt', 'mind'],
 ['get', 'boss', 'make', 'like', 'go'],
 ['least', 'number', 'expect', 'main', 'waste'],
 ['game', 'play', 'get', 'fun', 'boss'],
 ['fun', 'friend', 'really', 'multiplayer', 'server'],
 ['game', 'play', 'get', 'time', 'hour'],
 ['unicorn', 'shoot', 'gun', 'basically', 'fly']]

In [32]:
import pyLDAvis as vis

lda_vis_data = ctm.get_ldavis_data_format(tp.vocab, training_dataset, n_samples=10)

ctm_pd = vis.prepare(**lda_vis_data)
vis.display(ctm_pd)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

quantitative evaluation

like calulating u_mass, u_npmi score

or inversed RBO (diversity metrics)

In [40]:
def _get_topics(ctm, k=10):
    return ctm.get_topic_lists(k)

def _get_topic_word_metrix(ctm):
    return ctm.get_topic_word_distribution()

# ref: https://contextualized-topic-models.readthedocs.io/en/latest/readme.html (go to the section: Mono-Lingual Topic Modeling)
# testing_dataset = qt.transform(text_for_contextual=testing_text_for_contextual, text_for_bow=testing_text_for_bow)
# # n_sample how many times to sample the distribution (see the doc)
# ctm.get_doc_topic_distribution(testing_dataset, n_samples=20) # returns a (n_documents, n_topics) matrix with the topic distribution of each document
def _get_topic_document_metrix(ctm, dataset, n_samples=20):
    return ctm.get_doc_topic_distribution(dataset, n_samples=n_samples).T


topic_words = _get_topics(ctm, k=10)
topic_word_metrix = _get_topic_word_metrix(ctm)
topic_document_metrix = _get_topic_document_metrix(ctm, training_dataset, n_samples=20)

result_octis = {}
result_octis['topics'] = topic_words
result_octis['topic-word-metrix'] = topic_word_metrix
result_octis['topic-document-metrix'] = topic_document_metrix

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [34]:
topic_word_metrix.shape

(20, 2000)

In [35]:
topic_document_metrix.shape

(20, 75499)

In [36]:
from gensim import corpora

topic_words = _get_topics(ctm)
topics = ctm.get_predicted_topics(training_dataset, n_samples=20)

documents = pd.DataFrame({"Document": X,
                          "ID": range(len(X)),
                          "Topic": topics})

# remove documents which their topic contains 1<= words
# documents = documents[~documents['Topic'].isin(empty_topic_idxs)]

documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
# cleaned_docs = topic_model._preprocess_text(documents_per_topic.Document.values)

# bertopic_vectorizer = topic_model.vectorizer_model
# bertopic_analyzer = bertopic_vectorizer.build_analyzer()

# words = bertopic_vectorizer.get_feature_names_out()
# tokens = [bertopic_analyzer(doc) for doc in cleaned_docs]
texts = [doc.split() for doc in documents_per_topic.Document.values]

dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

/Users/michaelcheng/miniforge3/envs/fyp-test-tm/lib/python3.9/site-packages/gensim/matutils.py:22: DeprecationWarning: Please use `triu` from the `scipy.linalg` namespace, the `scipy.linalg.special_matrices` namespace is deprecated.
  from scipy.linalg.special_matrices import triu
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after par

In [37]:
# use Gensim to calculate coherence score

from gensim.models import CoherenceModel

coherence_model = CoherenceModel(topics=topic_words,
                                 texts=texts,
                                corpus=corpus,
                                dictionary=dictionary,
                                topn=10,
                                coherence='c_v')

In [38]:
print(coherence_model.get_coherence())

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

0.44762919445954436


In [41]:
coherence_model_npmi = CoherenceModel(topics=topic_words,
                                 texts=texts,
                                corpus=corpus,
                                dictionary=dictionary,
                                topn=10,
                                coherence='c_npmi')

print(coherence_model_npmi.get_coherence())

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

-0.02682839525228197


In [43]:
sys.path.append('../')
from eval_metrics import compute_inverted_rbo, compute_topic_diversity, compute_pairwise_jaccard_similarity

In [45]:
print('topic diversity:', compute_topic_diversity(result_octis, topk=10))
print('inverted rbo:', compute_inverted_rbo(result_octis, topk=10))
print('pairwise jaccard similarity:', compute_pairwise_jaccard_similarity(result_octis, topk=10))

topic diversity: 0.69
inverted rbo: 0.9561752377390601
pairwise jaccard similarity: 0.032942715405645175


Save the model

In [26]:
from datetime import datetime

ctm_model_path = Path(f'./ctm_model_{datetime.now().strftime("%Y%m%d_%H%M%S")}')
ctm.save(models_dir=ctm_model_path)

/root/miniforge3/envs/fyp-test-wsl-tm/lib/python3.9/site-packages/contextualized_topic_models/models/ctm.py:640: Warning: This is an experimental feature that we has not been fully tested. Refer to the following issue:https://github.com/MilaNLProc/contextualized-topic-models/issues/38
  warnings.warn(


load the model

In [29]:
ctm = CombinedTM(bow_size=len(tp.vocab), contextual_size=768, num_epochs=100, n_components=50)

model_datetime = datetime(2023, 1, 11, 19, 5, 43)
ctm_model_path

ctm.load(ctm_model_path.joinpath("contextualized_topic_model_nc_20_tpm_0.0_tpv_0.95_hs_prodLDA_ac_(100, 100)_do_softplus_lr_0.2_mo_0.002_rp_0.99"), epoch=9)

/root/miniforge3/envs/fyp-test-wsl-tm/lib/python3.9/site-packages/contextualized_topic_models/models/ctm.py:669: Warning: This is an experimental feature that we has not been fully tested. Refer to the following issue:https://github.com/MilaNLProc/contextualized-topic-models/issues/38
  warnings.warn(


In [ ]:
ctm.get_topic_lists(5)

[['disappointed', 'ingame', 'ex', 'experiance', 'success'],
 ['skyrim', 'ign', 'beating', 'scale', 'simulator'],
 ['go', 'kill', 'house', 'stuff', 'get'],
 ['buy', 'play', 'friend', 'friends', 'hour'],
 ['shame', 'unfortunately', 'lastly', 'initially', 'roughly'],
 ['rpg', 'minecraft', 'exploration', 'building', 'adventure'],
 ['unicorn', 'bunny', 'shoot', 'fish', 'lord'],
 ['ever', 'penny', 'best', 'definetly', 'addict'],
 ['assure', 'stress', 'custom', 'prefix', 'mid'],
 ['free', 'release', 'value', 'price', 'pay'],
 ['alright', 'guess', 'ok', 'neat', 'okay'],
 ['mac', 'please', 'fix', 'cant', 'crash'],
 ['average', 'frustrating', 'ingame', 'satisfaction', 'overwhelm'],
 ['terrarium', 'boss', 'minecraft', 'item', 'world'],
 ['collector', 'storage', 'favor', 'express', 'abandon'],
 ['scroller', 'platformer', 'retro', 'action', 'side'],
 ['game', 'play', 'time', 'new', 'one'],
 ['game', 'terrarium', 'minecraft', 'many', 'people'],
 ['lastly', 'harsh', 'enchantment', 'vs', 'sport'],
 ['